In [1]:
import torch
import torchvision
from torchvision.models import ResNet18_Weights

# 1) Load a pretrained model (already trained; we're not training here)
model = torchvision.models.resnet18(weights=ResNet18_Weights.DEFAULT).eval()

# 2) Dummy input shape must match the model's expectation
dummy_input = torch.randn(1, 3, 224, 224)

# 3) Export to ONNX: this freezes the graph + weights into a portable file
torch.onnx.export(
    model,
    dummy_input,
    "resnet18.onnx",
    input_names=["input"],       # name used later in TVM runtime
    output_names=["output"],
    opset_version=18,            # operator set version (compatibility)
    do_constant_folding=True,    # fold constants for small optimizations
    # dynamic_axes={"input": {0: "batch"}, "output": {0: "batch"}},  # optional
)

print("Wrote resnet18.onnx")

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /home/vscode/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100.0%


[torch.onnx] Obtain model graph for `ResNet([...]` with `torch.export.export(..., strict=False)`...
[torch.onnx] Obtain model graph for `ResNet([...]` with `torch.export.export(..., strict=False)`... ✅
[torch.onnx] Run decomposition...
[torch.onnx] Run decomposition... ✅
[torch.onnx] Translate the graph into ONNX...
[torch.onnx] Translate the graph into ONNX... ✅
Wrote resnet18.onnx


In [ ]:
import onnx
import tvm
from tvm import relay
from tvm.contrib import graph_executor

# Load the ONNX model
onnx_model = onnx.load("resnet18.onnx")

# Input shape dictionary. Ensure "input" matches the actual input name in your ONNX model.
# You can check this using onnx.load("resnet18.onnx").graph.input[0].name
input_name = "input" # Common input name, but verify if your model uses a different one
shape_dict = {input_name: (1, 3, 224, 224)}

# Import the ONNX model to TVM Relay format
# The error in your traceback (from_onnx(onnx_model, shape_dict)) implies onnx.mapping was called internally.
# Upgrading onnx should fix this if it's a version incompatibility.
mod, params = relay.frontend.from_onnx(onnx_model, shape_dict)

# Set the compilation target. "llvm" for CPU, "cuda" for NVIDIA GPUs.
target = "llvm" # or "cuda" if you have a GPU and CUDA installed

# Compile the model with TVM
with tvm.transform.PassContext(opt_level=3):
    lib = relay.build(mod, target=target, params=params)

# Save the compiled TVM artifacts
# These files can then be loaded by the TVM runtime for inference.
lib.export_library("deploy_lib_cpu.so")
with open("deploy_graph.json", "w") as f:
    f.write(lib.get_graph_json())
with open("deploy_params.params", "wb") as f:
    f.write(tvm.runtime.save_param_dict(lib.get_params()))

print("Compiled and exported TVM artifacts:")
print(" - deploy_lib_cpu.so")
print(" - deploy_graph.json")
print(" - deploy_params.params")

ImportError: Unable to import onnx which is required No module named 'onnx.mapping'